In [1]:
from mne.io import read_raw_edf
import mne
import pandas as pd

In [2]:
def find_channel(ch, chs_to_find):
    for ch_to_find in chs_to_find:
        if ch_to_find.lower() in ch.lower():
            return ch_to_find
    return ch

def add_montage(raw, ch_map=None):
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
    if ch_map:
        ten_twenty_montage.ch_names = [ch_map[ch] if ch in ch_map else ch
                                       for ch in ten_twenty_montage.ch_names]
    raw.set_montage(ten_twenty_montage)
    return raw

In [3]:
sample = read_raw_edf('test2.edf')

Extracting EDF parameters from C:\Users\Пользователь\Desktop\dev\test2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [4]:
chs_to_use = ['C3', 'C4', 'Cz', 'F3', 'F4', 'F7', 'F8', 'Fp1', 'Fp2', 'Fz',
               'O1', 'O2', 'P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6']

In [5]:
ch_map = {ch: find_channel(ch, chs_to_find=chs_to_use) for ch in sample.ch_names}

In [6]:
sample.rename_channels(ch_map);

In [7]:
sample.pick_channels(chs_to_use);

In [8]:
sample = add_montage(sample, ch_map=None)

In [9]:
mont = sample.get_montage()

### Сохраняем семпл

In [10]:
pd.DataFrame(sample.get_data()).to_csv('test2.csv', index=False, header=False)

In [11]:
with open('test2.xyz', 'w') as f:
    for i, (key, loc) in enumerate(mont.get_positions()['ch_pos'].items()):
        f.write(f'{i}\t{loc[0]}\t{loc[1]}\t{loc[2]}\t{key}\n')